In [4]:
import tensorflow as tf

from cfg import MAX_CAPTCHA, CHAR_SET_LEN, tb_log_path, save_model
from cnn_sys import crack_captcha_cnn, Y, keep_prob, X
from data_iter import get_next_batch


def train_crack_captcha_cnn():
    """
    训练模型
    :return:
    """
    output = crack_captcha_cnn()
    predict = tf.reshape(output, [-1, MAX_CAPTCHA, CHAR_SET_LEN])  # 36行，4列
    label = tf.reshape(Y, [-1, MAX_CAPTCHA, CHAR_SET_LEN])

    max_idx_p = tf.argmax(predict, 2)  # shape:batch_size,4,nb_cls
    max_idx_l = tf.argmax(label, 2)
    correct_pred = tf.equal(max_idx_p, max_idx_l)

    with tf.name_scope('my_monitor'):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predict, labels=label))
        # loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=output, labels=Y))
    tf.summary.scalar('my_loss', loss)
    # 最后一层用来分类的softmax和sigmoid有什么不同？

    # optimizer 为了加快训练 learning_rate应该开始大，然后慢慢衰
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

    with tf.name_scope('my_monitor'):
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    tf.summary.scalar('my_accuracy', accuracy)

    saver = tf.train.Saver()  # 将训练过程进行保存

    sess = tf.InteractiveSession(
        config=tf.ConfigProto(
            log_device_placement=False
        )
    )

    sess.run(tf.global_variables_initializer())
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter(tb_log_path, sess.graph)

    step = 0
    while True:
        batch_x, batch_y = get_next_batch(64)  # 64
        _, loss_ = sess.run([optimizer, loss], feed_dict={X: batch_x, Y: batch_y, keep_prob: 0.95})
        print(step, 'loss:\t', loss_)

        step += 1

        # 每2000步保存一次实验结果
        if step % 10 == 0:
            saver.save(sess, save_model, global_step=step)

        # 在测试集上计算精度
        if step % 50 != 0:
            continue

        # 每50 step计算一次准确率，使用新生成的数据
        batch_x_test, batch_y_test = get_next_batch(256)  # 新生成的数据集个来做测试
        acc = sess.run(accuracy, feed_dict={X: batch_x_test, Y: batch_y_test, keep_prob: 1.})
        print(step, 'acc---------------------------------\t', acc)

        # 终止条件
        if acc > 0.98:
            break

        # 启用监控 tensor board
        summary = sess.run(merged, feed_dict={X: batch_x_test, Y: batch_y_test, keep_prob: 1.})
        writer.add_summary(summary, step)


if __name__ == '__main__':
    train_crack_captcha_cnn()
    print('end')
    pass

验证码文本最长字符数 4
CHAR_SET_LEN: 36
model_path: C:\Users\josef\work/crack/model\crack_capcha.model
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

0 loss:	 3.59312
1 loss:	 3.60978
2 loss:	 3.5974
3 loss:	 3.58277
4 loss:	 3.59946
5 loss:	 3.58756
6 loss:	 3.59708
7 loss:	 3.59111
8 loss:	 3.59276
9 loss:	 3.57992
10 loss:	 3.57766
11 loss:	 3.59156
12 loss:	 3.59725
13 loss:	 3.58053
14 loss:	 3.58499
15 loss:	 3.58116
16 loss:	 3.59426
17 loss:	 3.59667
18 loss:	 3.59049
19 loss:	 3.58065
20 loss:	 3.5817
21 loss:	 3.58572
22 loss:	 3.59832
23 loss:	 3.58262
24 loss:	 3.5843
25 loss:	 3.58161
26 loss:	 3.58732
27 loss:	 3.58559
28 loss:	 3.58852
29 loss:	 3.60018
30 loss:	 3.5866
31 loss:	 3.58182
32 loss:	 3.58183


KeyboardInterrupt: 